In [9]:
import importData as import_data
import torch
import torch.nn as nn
import torch.optim as optim
import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
import numpy as np
id = import_data.importData()

if   torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {device}")

# Define the model
class CNNLSTMModel(nn.Module):
    def __init__(self, max_seq_length):
        super(CNNLSTMModel, self).__init__()
        self.conv = nn.Conv2d(9, 128, kernel_size=(3, 1))
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=(2, 1))  # Only pool along the height dimension
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(12672, 128)  # Adjusted dimensions after conv and pool
        self.repeat = nn.Linear(128, 128 * max_seq_length)
        self.lstm = nn.LSTM(128, 64, batch_first=True)
        self.fc_out = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.relu(self.fc(x))
        x = self.repeat(x)
        x = x.view(-1, max_seq_length, 128)  # Reshape to (batch_size, seq_length, input_size)
        x, _ = self.lstm(x)
        x = self.fc_out(x)
        return x
    

testuuid = "98006881-B2F0-4B6B-94CF-A21ABE69F21B"



d_test, t_test = id.import_data(testuuid, 0.02, 100, 2)[:100]
print(np.shape(d_test))
max_seq_length = t_test.shape[1]
# d_test, t_test = d_test[:100], t_test[:100]

new_data = torch.tensor(d_test, dtype=torch.float32).to(device)

# Load the saved model
model = CNNLSTMModel(max_seq_length).to(device)
model.load_state_dict(torch.load("torch_model_2.pth"))
model.eval()

# Run the model on the new dataset
with torch.no_grad():
    outputs = model(new_data)

# Convert the outputs to a NumPy array
outputs = outputs.cpu().numpy()

# Print the first few predictions
print("Model predictions on new dataset:")
print(outputs)

Using device: mps


/Users/thomas/Development/AATraining/importData.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_raw_df["time"] = trimmed_raw_df["time"].apply(lambda x: x - timestamps_df["startTime"][0])
/Users/thomas/Development/AATraining/importData.py:172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_time_dataset["time"] = start_time_dataset["time"] - start_time


(6803, 9, 200, 1)
Model predictions on new dataset:
[[[0.00674782]]

 [[0.00719797]]

 [[0.00790116]]

 ...

 [[0.00123977]]

 [[0.00096581]]

 [[0.00111672]]]


In [11]:
print(torch.cuda.is_available())
print(torch.__version__)

False
2.6.0
